# Heuristic Algorithms

A **heuristic** is a procedure that determines near-optimal solutions to an optimization problem.

- Sometimes it can find the optimal solution, but it cannot prove its optimality.
- Heuristics are widely used in industry!

# Traveling Salesman Problem (TSP) – Heuristics Overview

## Summary

The TSP seeks the shortest possible route that visits each city exactly once and returns to the origin city. Heuristic methods are often used to find good (but not always optimal) solutions efficiently.



## 1. Construction Heuristics

- **Nearest Neighbor:**  
  Start at a city, repeatedly visit the nearest unvisited city.

- **Cheapest Insertion:**  
  Start with a small tour and repeatedly insert the city that results in the smallest increase in total distance.

- **...**  
  (Other construction heuristics exist.)



## 2. Improvement Heuristics (Local Search)

- **2-opt:**  
  Iteratively remove two edges and reconnect the two paths in a different way to reduce the tour length.

- **3-opt:**  
  Similar to 2-opt, but removes and reconnects three edges for potentially better improvements.


## Key Idea

- **Construct, then improve:**  
  Build an initial solution using a construction heuristic, then refine it using improvement heuristics.



## Common Wisdom

- **Solution quality depends largely on improvement heuristics!**



## Improvement Details

- **Solution representation:**  
  Typically as a sequence of nodes (cities).

- **Definition of neighbor:**  
  For example, in 2-opt, neighbors are tours that can be reached by swapping two edges.

- **Search strategy:**  
  Often, the best (optimal) neighbor is chosen at each step.



## Caution: Local Optimal

- **Local minimum:**  
  A solution where no small change improves the tour, but it may not be the global best.

- **Global minimum:**  
  The absolute shortest possible tour.

- **How to escape from local minimum?**  
  Advanced techniques (like simulated annealing, tabu search, or random restarts) can help escape local minima and find better solutions.

In [ ]:
# 1. 最近邻法（Nearest Neighbor）
import numpy as np


def nearest_neighbor_tsp(distance_matrix, start=0):
    n = len(distance_matrix)
    visited = [False] * n
    tour = [start]
    visited[start] = True
    current = start

    for _ in range(n - 1):
        next_city = np.argmin(
            [
                distance_matrix[current][j] if not visited[j] else np.inf
                for j in range(n)
            ]
        )
        tour.append(next_city)
        visited[next_city] = True
        current = next_city

    tour.append(start)  # 返回起点
    return tour


# 2. 最便宜插入法（Cheapest Insertion）
def cheapest_insertion_tsp(distance_matrix):
    n = len(distance_matrix)
    unvisited = set(range(n))
    tour = [0, np.argmin(distance_matrix[0, 1:]) + 1, 0]
    unvisited -= set(tour)
    while unvisited:
        best_increase = np.inf
        best_city = None
        best_pos = None
        for city in unvisited:
            for i in range(1, len(tour)):
                increase = (
                    distance_matrix[tour[i - 1], city]
                    + distance_matrix[city, tour[i]]
                    - distance_matrix[tour[i - 1], tour[i]]
                )
                if increase < best_increase:
                    best_increase = increase
                    best_city = city
                    best_pos = i
        tour.insert(best_pos, best_city)
        unvisited.remove(best_city)
    return tour


# 3. 2-opt 局部搜索
def two_opt(tour, distance_matrix):
    n = len(tour)
    improved = True
    while improved:
        improved = False
        for i in range(1, n - 2):
            for j in range(i + 1, n - 1):
                if j - i == 1:
                    continue
                if (
                    distance_matrix[tour[i - 1], tour[i]]
                    + distance_matrix[tour[j], tour[j + 1]]
                    > distance_matrix[tour[i - 1], tour[j]]
                    + distance_matrix[tour[i], tour[j + 1]]
                ):
                    tour[i : j + 1] = reversed(tour[i : j + 1])
                    improved = True
    return tour


# 4. 3-opt 局部搜索
def three_opt(tour, distance_matrix):
    def cost(tour):
        return sum(distance_matrix[tour[i], tour[i + 1]] for i in range(len(tour) - 1))

    n = len(tour)
    improved = True
    while improved:
        improved = False
        for i in range(1, n - 4):
            for j in range(i + 1, n - 2):
                for k in range(j + 1, n):
                    new_tours = [
                        tour[:i] + tour[i:j][::-1] + tour[j:k][::-1] + tour[k:],
                        tour[:i] + tour[j:k] + tour[i:j] + tour[k:],
                        tour[:i] + tour[j:k][::-1] + tour[i:j] + tour[k:],
                        tour[:i] + tour[i:j][::-1] + tour[j:k] + tour[k:],
                    ]
                    for new_tour in new_tours:
                        if cost(new_tour) < cost(tour):
                            tour = new_tour
                            improved = True
    return tour


# 示例数据
distance_matrix = np.array([[0, 2, 9, 10], [1, 0, 6, 4], [15, 7, 0, 8], [6, 3, 12, 0]])

# 调用示例
tour_nn = nearest_neighbor_tsp(distance_matrix)
tour_ci = cheapest_insertion_tsp(distance_matrix)
tour_2opt = two_opt(tour_nn.copy(), distance_matrix)
tour_3opt = three_opt(tour_nn.copy(), distance_matrix)

print("最近邻法:", tour_nn)
print("最便宜插入法:", tour_ci)
print("2-opt优化:", tour_2opt)
print("3-opt优化:", tour_3opt)

最近邻法: [0, 1, 3, 2, 0]
最便宜插入法: [0, 1, 2, 3, 0]
2-opt优化: [0, 2, 3, 1, 0]
3-opt优化: [0, 1, 3, 2, 0]


# Simulated Annealing Algorithm

## The Metropolis Criterion

The probability of accepting a worse solution is based on thermodynamics:

$$p = e^{-\Delta E / kT}$$

where:  
- $\Delta E$: Change in objective value  
- $k$: Positive constant  
- $T$: Current temperature  
- $p \in (0,1)$ when $\Delta E > 0$


## Algorithm Parameters

- Solution space $\Omega$  
- Objective function $f(\omega)$  
- Initial temperature $T = t_0 \geq 0$  
- Cooling schedule $t_k$  
- Repetition schedule $M_k$


## Algorithm Steps

1. Initialize solution $\omega \in \Omega$  
2. Set temperature counter $k = 0$  
3. While stopping criterion not met:  
   - For $m = 1$ to $M_k$:  
     - Generate neighbor $\omega' \in N(\omega)$  
     - Calculate $\Delta = f(\omega') - f(\omega)$  
     - If $\Delta \leq 0$, accept $\omega \leftarrow \omega'$  
     - Else, accept $\omega \leftarrow \omega'$ with probability $e^{-\Delta / t_k}$  
   - Update temperature: $k \leftarrow k + 1$


## Key Features

- Accepts worse solutions with decreasing probability  
- Temperature controls exploration vs. exploitation  
- Converges to local optima as $T \to 0$

In [ ]:
import random
import numpy as np
import math

num_city = 30  # 城市总数
initial_t = 120  # 初始温度
lowest_t = 0.001  # 最低温度
M = 150  # 当连续多次都不接受新的状态，开始改变温度
iteration = 500  # 设置迭代次数

location = np.loadtxt("city_location.txt")


# ==========================================
# 对称矩阵，两个城市之间的距离
def distance_p2p_mat():
    dis_mat = []
    for i in range(num_city):
        dis_mat_each = []
        for j in range(num_city):
            dis = math.sqrt(
                pow(location[i][0] - location[j][0], 2)
                + pow(location[i][1] - location[j][1], 2)
            )
            dis_mat_each.append(dis)
        dis_mat.append(dis_mat_each)
    # print(dis_mat)
    return dis_mat


# 计算所有路径对应的距离
def cal_newpath(dis_mat, path):
    dis = 0
    for j in range(num_city - 1):
        dis = dis_mat[path[j]][path[j + 1]] + dis
    dis = dis_mat[path[num_city - 1]][path[0]] + dis  # 回家
    return dis


# ==========================================
# 点对点距离矩阵
dis_mat = distance_p2p_mat()
# 初始路径
path = list(range(num_city))
# 初始距离
dis = cal_newpath(dis_mat, path)
# 初始温度
t_current = initial_t

while t_current > lowest_t:  # 外循环，改变温度
    count_m = 0  # M的计数
    count_iter = 0  # 迭代次数计数
    while (
        count_m < M and count_iter < iteration
    ):  # 内循环，连续多次不接受新的状态或者是迭代多次,跳出内循环
        i = 0
        j = 0
        while i == j:  # 防止随机了同一城市
            i = random.randint(0, num_city - 1)
            j = random.randint(0, num_city - 1)
        path_new = path.copy()
        path_new[i], path_new[j] = (
            path_new[j],
            path_new[i],
        )  # 任意交换两个城市的位置,产生新解
        # 计算新解的距离
        dis_new = cal_newpath(dis_mat, path_new)
        # 求差
        dis_delta = dis_new - dis
        # 取0-1浮点随机数
        rand = random.random()
        # 计算指数函数的值
        exp_d = math.exp(-dis_delta / t_current)
        # 选择
        if dis_delta < 0:
            path = path_new
            dis = dis_new
        elif exp_d > rand:
            path = path_new
            dis = dis_new
        else:
            count_m = count_m + 1
        count_iter = count_iter + 1
    t_current = 0.99 * t_current  # 改变温度
# 外循环结束
dis_min = dis
path_min = path
print("最短距离：", dis_min)
print("最短路径：", path_min)

最短距离： 458.39698234162626
最短路径： [29, 22, 21, 11, 12, 3, 8, 2, 17, 18, 19, 20, 9, 10, 6, 7, 13, 14, 23, 24, 25, 26, 27, 28, 15, 16, 1, 0, 5, 4]


# Heuristic Algorithm for Facility Location Problem

## Problem Description

The Facility Location Problem aims to determine which facilities to open and how to assign customers to these facilities in order to minimize the total cost, including both fixed opening costs and variable transportation costs.

- **Fixed cost**: The cost to open each facility.
- **Transportation cost**: The cost to serve each customer from an open facility.

## Heuristic Algorithm Idea

1. **Initialization**: All facilities are closed, and all customers are unassigned.
2. **Greedy Selection**: In each iteration, select the unopened facility with the lowest unit cost (total fixed cost plus transportation cost divided by the number of customers served), and assign all unassigned customers to it.
3. **Update**: Remove assigned customers from the unassigned set. Repeat until all customers are assigned.

## Pseudocode

1. For each unopened facility, calculate its unit cost.
2. Open the facility with the lowest unit cost and assign all unassigned customers to it.
3. Repeat until all customers are assigned.


In [ ]:
import numpy as np

m, n = 5, 8
fixed_cost = [100, 120, 90, 110, 95]
transport_cost = np.random.randint(10, 50, size=(m, n))

opened = [False] * m
assigned = [-1] * n
unassigned_customers = set(range(n))

while unassigned_customers:
    best_facility = -1
    best_unit_cost = float("inf")
    best_customers = []

    for i in range(m):
        if opened[i]:
            continue
        customers = list(unassigned_customers)
        costs = [transport_cost[i][j] for j in customers]
        total_cost = fixed_cost[i] + sum(costs)
        unit_cost = total_cost / (len(customers) + 1e-6)
        if unit_cost < best_unit_cost:
            best_unit_cost = unit_cost
            best_facility = i
            best_customers = customers

    opened[best_facility] = True
    for j in best_customers:
        assigned[j] = best_facility
    unassigned_customers -= set(best_customers)

print("Opened facilities:", [i for i, open_ in enumerate(opened) if open_])
print("Customer assignments:", assigned)

Opened facilities: [4]
Customer assignments: [4, 4, 4, 4, 4, 4, 4, 4]
